In [0]:
!pip install sqlalchemy
!pip install setproctitle
!pip install ray[rllib]  # also recommended: ray[debug]
!pip uninstall -y pyarrow

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd  /content/drive/My\ Drive/Year\ 4\:\ Synthesis/NS162\:\ Statistical\ Mechanics/Renormalization\ Research\ Project/renormalization/supervised_convnet/t_2.269/81x81/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/renormalization/supervised_convnet/t_2.269/81x81


In [3]:
# %%capture
import train#, frozen
import sys
# sys.path.insert(0, "../../")
import supervised_convnet
import pickle
from collections import defaultdict
import numpy as np
import time
import torch

mode = "run"
run_mode = "multilayer_1channel"
save_loops = 50
use_cuda = True
results = []
conv_params = {}
filename = ""
def save_progress(results, conv_params, filename, epoch = 500):
    new_results = {}
    new_results["best_val_acc_hist"] = results
    new_results["conv_params"] = conv_params
    new_results["annotation"] = "bias + weights histogram saved"
    new_results["epoch"] = epoch
    # new_results["first_epo    ch_validate_accuracy_list"] = first_epoch_validate_accuracy_list
    with open(filename, "wb") as handle:
        pickle.dump(new_results, handle, protocol = pickle.HIGHEST_PROTOCOL)
"""
Default activation function: sigmoid
"""

# GPU for use in colab
def model_to_cuda(model):
    if use_cuda and torch.cuda.is_available():
        model = model.cuda()
    return model
  
if mode == "run":
    hidden_size = 10
    out_channels = 1
    num_hidden_layers = 1
    num_conv_layers = 3
    conv_params = defaultdict(list)
    

    if run_mode == "multilayer_1channel":
        """
        """
        out_channels = 1
        filename = "multilayer_1channel.pl"
        try:
            with open(filename, "rb") as handle:
                results = pickle.load(handle)
        except:
            results = []
        results = []
        for _ in range(500):
            model = supervised_convnet.MultiLayerConvNet(filter_size = 3, square_size = 3, \
                    hidden_size = hidden_size, out_channels = out_channels,
                    first_activation = "tanh", activation_func = "relu",
                    num_hidden_layers = num_hidden_layers, num_conv_layers = num_conv_layers, seed = time.time() + _)
            model = model_to_cuda(model)
            best_val_acc, param_dict = train.trainer(model = model, batch_size = 534, betas0= 1-0.009367812437484689, betas1=1- 0.0011636320996308597,
                                                    lr= 0.18602018785651322, n_epochs= 165, train_size= 2459, weight_decay= 0.0034972858295468723, use_cuda = use_cuda)
            results.append(best_val_acc)
            for i in range(3):
                conv_params[f"weight_{i}"].append(param_dict[f"conv_layers.{i}.weight"])
                conv_params[f"bias_{i}"].append(param_dict[f"conv_layers.{i}.bias"])
            if (_ % save_loops) == (0):
                save_progress(results, conv_params, filename, _)


save_progress(results, conv_params, filename)



Output hidden; open in https://colab.research.google.com to view.

In [0]:
sys.path.pop(0)

'../../'

In [4]:
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [5]:
supervised_convnet.__file__

'/content/drive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/renormalization/supervised_convnet/t_2.269/81x81/supervised_convnet.py'

In [0]:
!cat supervised_convnet.py




import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import math
import numpy as np
import sys
import time

class SupervisedConvNet(nn.Module):
    def __init__(self, filter_size, square_size, hidden_size, num_hidden_layers,
                first_activation = "tanh", activation_func = "sigmoid",
                out_channels = 1, seed = time.time()):
        """
        Arguments:
        filter_size ~ size of the convolution kernel (3 x 3)
        square size ~ how many strides of convolution in the input
        """
        torch.manual_seed(seed)
        super(SupervisedConvNet, self).__init__()
        self.filter_size = filter_size
        self.square_size = square_size
        self.hidden_size = hidden_size
        self.out_channels = out_channels
        if first_activation == "tanh":
            self.first_activation = torch.tanh
        elif first_act